In [1]:
import numpy as np
import pandas as pd
import imageio
import h5py
import pickle

from tqdm import tqdm
from xqdm import xqdm
from pathlib import Path
import matplotlib.pyplot as plt
import re

import keras.layers as L
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard

Using TensorFlow backend.


In [2]:
I = L.Input(shape=(6,))
x = L.Dense(12, activation='tanh')(I)
x = L.Dense(24, activation='tanh')(x)
x = L.Dropout(0.2)(x)
x = L.Dense(48, activation='tanh')(x)
x = L.Dropout(0.2)(x)
x = L.Dense(1, activation='linear')(x)

model = Model(inputs=I, outputs=x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 12)                84        
_________________________________________________________________
dense_2 (Dense)              (None, 24)                312       
_________________________________________________________________
dropout_1 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 48)                1200      
_________________________________________________________________
dropout_2 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 49  

In [4]:
model.compile('adam', loss='mean_squared_error')

In [5]:
with open('data/mlp300_yaw.pkl', 'rb') as f:
    X_train, X_test, Y_train, Y_test = pickle.load(f)

#X = np.stack(X, axis=0)
#Y = np.stack(Y, axis=0)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(178927, 6) (178544, 6) (178927,) (178544,)


In [6]:
name = '0008'
checkpoint_dir = Path('checkpoint') / name
checkpoint_dir.mkdir(parents=True, exist_ok=True)

In [7]:
checkpoint_path = str(checkpoint_dir / ('ckpt-%s-epoch{epoch:04d}-val_loss{val_loss:.4f}.hdf5'%name))

callbacks = [
    ReduceLROnPlateau(verbose=1),
    EarlyStopping(patience=10, verbose=1),
    ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True),
    TensorBoard(log_dir=checkpoint_dir)
]

In [8]:
model.fit(X_train, Y_train, batch_size=64, epochs=100, validation_data=(X_test, Y_test), verbose=1, shuffle=False, callbacks=callbacks)


Train on 178927 samples, validate on 178544 samples


Epoch 1/100
178927/178927 [==============================] - 2s 13us/step - loss: 0.0118 - val_loss: 0.0028

Epoch 00001: val_loss improved from inf to 0.00285, saving model to checkpoint/0008/ckpt-0008-epoch0001-val_loss0.0028.hdf5

Epoch 2/100
178927/178927 [==============================] - 2s 13us/step - loss: 0.0077 - val_loss: 0.0031

Epoch 00002: val_loss did not improve from 0.00285
Epoch 3/100
178927/178927 [==============================] - 2s 13us/step - loss: 0.0073 - val_loss: 0.0027

Epoch 00003: val_loss improved from 0.00285 to 0.00269, saving model to checkpoint/0008/ckpt-0008-epoch0003-val_loss0.0027.hdf5
Epoch 4/100
178927/178927 [==============================] - 2s 12us/step - loss: 0.0071 - val_loss: 0.0028

Epoch 00004: val_loss did not improve from 0.00269
Epoch 5/100
178927/178927 [==============================] - 2s 12us/step - loss: 0.0068 - val_loss: 0.0028

Epoch 00005: val_loss did not improve from 0.